# Section 3
## Read in database credentials
Below the credentials for connecting to the database are read into variables by extracting the lines from the local file. The local file is not included in the repo.

In [2]:
db_name = ""
db_user = ""
db_pass = ""
db_host = ""
with open("database_credentials.txt") as f:
    db_name = f.readline().strip()
    db_user = f.readline().strip()
    db_pass = f.readline().strip()
    db_host = f.readline().strip()

## Shorthand connect to database
This method will just return a new database connection with the default credentials made available above.

In [3]:
import pymysql as pms

In [4]:
def get_connect():
    """
    Returns a database connection object using the default parameters
    specified in the database_credentials file read in at the start of
    this notebook.
    """
    return pms.connect(host=db_host, user=db_user, passwd=db_pass, db=db_name);

## Test connection
The next code segment tests to ensure that the database connection is working properly.

In [5]:
try:
    con = get_connect()
    print("Successfully connected")
finally:
    if con:
        print("Closing connection")
        con.close()

Successfully connected
Closing connection


In [6]:
def get_connect():
    """
    Returns a database connection object using the default parameters
    specified in the database_credentials file read in at the start of
    this notebook.
    """
    return pms.connect(host=db_host, user=db_user, passwd=db_pass, db=db_name);

## Shorthand query execution and output
The method below accepts a single parameter (expected query), executes the parameter as a SQL query, and outputs the results. The connection is closed before the function terminates.

In [12]:
def execute_sql_output_result(query_string):
    """
    Given the query_string parameter, this function connects to the database, executes
    the query, outputs the result, and closes the connection.
    """
    try:
        con = get_connect()
        with con.cursor() as cur:
            #If the query_string is a single string, execute the string
            if type(query_string) == str:
                cur.execute(query_string)
                result = cur.fetchall()
                print("=== {} RESULTS ===".format(len(result)))
                #Column names
                print(" ".join([i[0] for i in cur.description]))
                #Results
                for i in range(len(result)):
                    print("{}: {}".format(i, result[i]))
                print()
    finally:
        if con:
            con.close()

## Group functions
A group function operates on several rows and returns a single result.
### MAX, MIN, and SUM

In [14]:
execute_sql_output_result("""
    SELECT MAX(sal) AS MaxSal, MIN(sal) AS MinSal, SUM(sal) AS SumSal FROM EMP;
""")
execute_sql_output_result("""
    SELECT MAX(sal) AS MaxManagerSal FROM emp WHERE job='MANAGER';
""")

=== 1 RESULTS ===
MaxSal MinSal SumSal
0: (5000.0, 800.0, 29025.0)

=== 1 RESULTS ===
MaxManagerSal
0: (2975.0,)



### COUNT

In [16]:
execute_sql_output_result("""
    SELECT COUNT(ename) AS EmployeeCount, COUNT(comm) AS EmployeesWithCommissionb FROM emp;
""")

=== 1 RESULTS ===
EmployeeCount EmployeesWithCommission
0: (14, 4)



### Averaging values

In [22]:
execute_sql_output_result("""
    SELECT CONCAT('$', FORMAT(SUM(sal) / COUNT(sal), 2)) AS AverageSalary,
        CONCAT('$', FORMAT(AVG(sal), 2)) AS AverageNativeSalary FROM emp;
""")

=== 1 RESULTS ===
AverageSalary AverageNativeSalary
0: ('$2,073.21', '$2,073.21')



## GROUP BY and HAVING clauses
The `GROUP BY` clause allows you to perform multi row function computation on a distinct value basis.

In [26]:
execute_sql_output_result("""
    SELECT job, AVG(sal) FROM emp GROUP BY job
""")

execute_sql_output_result("""
    SELECT job, min(sal) FROM emp GROUP BY job
""")

=== 5 RESULTS ===
job AVG(sal)
0: ('ANALYST', 3000.0)
1: ('CLERK', 1037.5)
2: ('MANAGER', 2758.333333)
3: ('PRESIDENT', 5000.0)
4: ('SALESMAN', 1400.0)

=== 5 RESULTS ===
job min(sal)
0: ('ANALYST', 3000.0)
1: ('CLERK', 800.0)
2: ('MANAGER', 2450.0)
3: ('PRESIDENT', 5000.0)
4: ('SALESMAN', 1250.0)



### HAVING
`HAVING` can be used to specify a certain criteria in conjunction with `GROUP BY`.

In [37]:
execute_sql_output_result("""
    SELECT job, AVG(sal) FROM emp GROUP BY job HAVING count(*) = 4 ORDER BY job DESC;
""")

execute_sql_output_result("""
    SELECT deptno FROM emp GROUP BY deptno HAVING count(*) > 3;
""")

=== 2 RESULTS ===
job AVG(sal)
0: ('SALESMAN', 1400.0)
1: ('CLERK', 1037.5)

=== 2 RESULTS ===
deptno
0: (20,)
1: (30,)

